In [5]:
pip install Keras


     |████████████████████████████████| 378kB 1.4MB/s eta 0:00:01    |█▊                              | 20kB 853kB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 28.8MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 15.2MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 276kB 2.7MB/s eta 0:00:01
  Running setup.py install for pyyaml ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install torch

     |████████████████████████████████| 80.6MB 2.9MB/s eta 0:00:01     |██████▌                         | 16.3MB 2.6MB/s eta 0:00:26
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from math import nan
#from future.utils import iteritems
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest
import numpy as np

In [24]:
data = pd.read_csv("ner_dataset.csv") 
data.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 0: invalid start byte

In [23]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

NameError: name 'data' is not defined

In [ ]:
maxlen = max([len(s) for s in sentences])

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)